In [17]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import os
from datetime import datetime
import requests
from bs4 import BeautifulSoup


from src.data import metadata

In [2]:
metadata_df = metadata.build_df()

In [3]:
metadata_df

,KNUMBER,APPLICANT,CONTACT,STREET1,STREET2,CITY,STATE,COUNTRY_CODE,ZIP,POSTAL_CODE,...,DECISION,REVIEWADVISECOMM,PRODUCTCODE,STATEORSUMM,CLASSADVISECOMM,SSPINDICATOR,TYPE,THIRDPARTY,EXPEDITEDREVIEW,DEVICENAME
0,K760001,"ZIMMER, INC.",NaN,"4221 Richmond Rd., N.W.",NaN,Walker,MI,US,49534,49534,...,SESE,PM,NaN,NaN,NaN,NaN,Traditional,N,NaN,ARCH SUPPORT (ARCH AID)
1,K760002,"ZIMMER, INC.",NaN,NaN,NaN,NaN,MO,US,NaN,NaN,...,SESE,PM,IQI,NaN,PM,NaN,Traditional,N,NaN,KNEE AID
2,K760003,"ZIMMER, INC.",NaN,803 N. Front St. Suite 3,NaN,McHenry,IL,US,60050,60050,...,SESE,PM,ITG,NaN,PM,NaN,Traditional,N,NaN,CAST MATERIAL (WICKET STOCKINETTE)
3,K760004,STEWART-NAUMANN LABORATORIES,NaN,803 N. Front St. Suite 3,NaN,McHenry,IL,US,60050,60050,...,SESE,HO,FMF,NaN,HO,NaN,Traditional,N,NaN,"SYRINGE, DISPOSABLE, ALL PLASTIC"
4,K760005,STEWART-NAUMANN LABORATORIES,NaN,803 N. Front St. Suite 3,NaN,McHenry,IL,US,60050,60050,...,SESE,HO,FMF,NaN,HO,NaN,Traditional,N,NaN,"SYRINGE, DISPOSABLE, GLASS & PLASTIC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,K955593,SHIPPERT MEDICAL TECHNOLOGIES CORP.,SARAH LAKE,"7002 SOUTH REVERE PARKWAY,",SUITE 60,ENGLEWOOD,CO,US,80112,80112,...,SESE,EN,EMX,Summary,EN,NaN,Traditional,N,NaN,EXPANDACELL ANATOMICAL POSTERIOR NASAL PACK W/...
23312,K955609,COLOPLAST CORP.,M. SYDNEY LILLY,1955 WEST OAK CIR.,NaN,MARIETTA,GA,US,30062-2249,30062-2249,...,SESE,GU,FAQ,Statement,GU,NaN,Traditional,N,NaN,CONVEEN URINE BAG - STERILE
23313,K955613,MISTEN MEDICAL CORP.,GEORGE W MURRAY,7220 STOUT RD.,NaN,GERMANTOWN,TN,US,38138,38138,...,SESE,EN,ETB,Statement,EN,NaN,Traditional,N,NaN,MISTEN MEDICAL PARTIAL OSSICULAR REPLACEMENT P...
23314,K955615,MISTEN MEDICAL CORP.,GEORGE W MURRAY,7220 STOUT RD.,NaN,GERMANTOWN,TN,US,38138,38138,...,SESE,EN,ETA,Statement,EN,NaN,Traditional,N,NaN,MISTEN MEDICAL TOTAL OSSICULAR REPLACEMENT PRO...


In [34]:
def fetch_summary_pdf(k_number, save_location='data/raw/pdf'):
    """Downloads a pdf of a summary given a k_number

    Fetches a pdf given a k number and saves it to a given directory. Note that many of these applications
    don't hav e summaries associated with them (especially the older ones.) If you enter a k_number value
    that doesn't have a summary this will raise an error.

    Args:
        k_number: (str) The k number of the document. This is the number that the FDA uses to identify these docs
        save_location: ( str, pathlike, default = data/raw/pdf')= The directory you want to save this in

    Returns:
        save_path (str) the filepath where the file was saved

    Raises:
        Exception: If there is no pdf accoiated with the k_number or the k_number cannot be found

    """
    try:
        url = metadata.find_pdf(k_number)
    except:
        raise FileExistsError("Could not retrive URL of pdf for accosiated k_number. It probably just doesn't exist")

    response = requests.get(url)
    filepath = "{}.pdf".format(k_number)
    save_path = os.path.join(save_location, filepath)
    with open(save_path, 'wb') as pdf:
        pdf.write(response.content)
    return save_path





SyntaxError: invalid syntax (<ipython-input-34-77029e2981b2>, line 1)

In [11]:
k_number = 'K183074'
url = 'https://www.accessdata.fda.gov/cdrh_docs/pdf{}/{}.pdf'.format(18, k_number)
response = requests.get(url)

In [13]:
response.content

'%PDF-1.4\n%����\n1 0 obj\n<<\n/Type /Catalog\n/Version /1.4\n/Pages 2 0 R\n>>\nendobj\n2 0 obj\n<<\n/Type /Pages\n/Kids [3 0 R 4 0 R 5 0 R 6 0 R 7 0 R 8 0 R 9 0 R 10 0 R 11 0 R 12 0 R\n13 0 R 14 0 R 15 0 R 16 0 R 17 0 R 18 0 R]\n/Count 16\n>>\nendobj\n3 0 obj\n<<\n/Annots [19 0 R 20 0 R]\n/Contents [21 0 R 22 0 R 23 0 R 24 0 R 25 0 R 26 0 R 27 0 R 28 0 R]\n/CropBox [0 0 612 792]\n/Group <<\n/CS /DeviceRGB\n/S /Transparency\n/Type /Group\n>>\n/MediaBox [0 0 612 792]\n/Parent 2 0 R\n/Resources <<\n/ExtGState <<\n/GS7 29 0 R\n/GS8 30 0 R\n>>\n/Font <<\n/F1 31 0 R\n>>\n/ProcSet [/PDF /Text /ImageB /ImageC /ImageI]\n/XObject <<\n/Image11 32 0 R\n>>\n>>\n/Rotate 0\n/StructParents 0\n/Tabs /S\n/Type /Page\n>>\nendobj\n4 0 obj\n<<\n/Annots [33 0 R 34 0 R 35 0 R 36 0 R 37 0 R 38 0 R 39 0 R 40 0 R 41 0 R 42 0 R\n43 0 R 44 0 R 45 0 R 46 0 R 47 0 R 48 0 R 49 0 R 50 0 R 51 0 R 52 0 R\n53 0 R]\n/Contents 54 0 R\n/CropBox [0 0 612 792]\n/Group <<\n/CS /DeviceRGB\n/S /Transparency\n/Type /Group\n>>\n

In [16]:
import PyPDF2


AttributeError: 'bytes' object has no attribute 'seek'

In [86]:
def find_pdf(k_number):
    """
    This function will take a k_number you provide and look on the FDA site to try to find a summary
    pdf to go along with it. If it finds a summary pdf it will return the link to it.
    Args:
        k_number: (str) the string of the k_number of the document you are looking for

    Returns: (str) url of the link to the pdf if found

    Raises:
        ValueError: This will raise an error if the k_number provided doesn't have a url associated with
                                    it on the FDA website
         Error: This will be thrown if the site doensn't have an associated summary document with it

    """


    root_url = 'https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/pmn.cfm?ID={}'
    response = requests.get(root_url.format(k_number))
    soup = BeautifulSoup(response.text)


    for x in soup.findAll('tr'):
        th= x.findNext('th')
        th_text = th.text.replace('\n', '').replace('\t', '')
        td = x.findNext('td')
        if th_text == 'summary':
            pdf_link = td.findNext('a', href=True)['href']
            return pdf_link

    raise Exception("Could not find PDF. This probably means that this application was submitted without a summary or that the robot killer finally found me. ")


what the fuck
http://www.accessdata.fda.gov/cdrh_docs/pdf18/K183074.pdf
